In [1]:
# Build Baseline dataset from all valid trials from Baseline sessions in the AFIRM and BAA classifications
# File structure from root_dir:
# {root_dir}/
# ├── {classification}/
# │   ├── {subject}/
# │   │   ├── {session}/
# │   │   │   ├── {trial}.c3d
# │   │   │   └── ...
# │   │   └── ...
# │   └── ...
# └── ...

root_dir = 'data'
classifications = ['AFIRM', 'BAA']
session = 'Baseline'

from utils import scandir_regex
from rat_trial import RatTrial
import os

file_pattern = f"{root_dir}/({'|'.join(classifications)})/.*/{session}/.*\\.c3d"
print(f"Searching for files with pattern: {file_pattern}")

baseline_files = scandir_regex(root_dir, file_pattern)
for file in baseline_files:
    print(f"Processing file: {file}")
    trial = RatTrial.from_c3d_file(file)
    # Validate
    if not trial.validate_trial():
        print(f"Invalid trial: {file}")
        continue
    trial.to_trc(file.replace('.c3d', '.trc'))
    if 'Static' in trial.name:
        trial.create_scaled_opensim(
            unscaled_model_path='opensim/Bilateral/FinalBilateral.osim',
            marker_set_path='opensim/Bilateral/bilateral_rodent_hindlimb_markerSet.xml',
            marker_file_name= os.path.basename(file).replace('.c3d', '.trc'),
            output_dir=os.path.dirname(file),
            scale_setup_path='opensim/Bilateral/generic_scale_setup_bilateral.xml'
        )


Searching for files with pattern: data/(AFIRM|BAA)/.*/Baseline/.*\.c3d
Processing file: data/BAA/BAA01/Baseline/Walk05.c3d
Processing file: data/BAA/BAA01/Baseline/Static01.c3d


2025-06-17 05:23:56.771 | INFO     | rat_trial:create_scaled_opensim:466 - Scaled model saved to /home/hudson/Rats_Python/data/BAA/BAA01/Baseline/Static01_scaled.osim
2025-06-17 05:23:56.783 | INFO     | rat_trial:create_scaled_opensim:466 - Scaled model saved to /home/hudson/Rats_Python/data/BAA/BAA01/Baseline/Static01_marker.osim


[info] Processing subject Static01...
[info] Step 1: Loading generic model
[info] Updating Model file from 40000 to latest format...
[info] Loaded model FinalBilateral from file /home/hudson/Rats_Python/opensim/Bilateral/FinalBilateral.osim
[info] Loading marker set from '/home/hudson/Rats_Python/data/BAA/BAA01/Baseline//home/hudson/Rats_Python/opensim/Bilateral/bilateral_rodent_hindlimb_markerSet.xml'.
[info] Updated markers in model FinalBilateral
[info] Step 2: Scaling generic model
[info] Loaded marker file /home/hudson/Rats_Python/data/BAA/BAA01/Baseline/Static01.trc (12 markers, 676 frames)
[info] Measurement 'spine'
[info] 	pair 0 (TAIL, SPL6): model = 0.0393357, experimental = 0.0352886
[info] 	overall scale factor = 0.897113
[info] Measurement 'pelvis_length'
[info] 	pair 0 (RASI, RHIP): model = 0.0299833, experimental = 0.0221034
[info] 	pair 1 (LASI, LHIP): model = 0.0299833, experimental = 0.0221036
[info] 	pair 2 (SPL6, TAIL): model = 0.0393357, experimental = 0.0352886
[i

In [ ]:
# Build pathology groups
classification = 'AFIRM'
NR = [
    "LGS09",
    "LGS10",
    "LGS13",
    "LGS14",
    "E17",
    "E20",
    "E21",
    "LGS11",
    "LGS12",
    "LGS15",
    "LGS16",
    "T05",
    "T06",
    "E16",
    "E18",
    "E19",
    "LGS08",
    "T08",
    "E08",
    "E12",
    "E13",
    "E14",
    "E15",
    "E26",
    "LGS05",
    "LGS06",
    "LGS07",
    "T07"
]
TEMR = [
    "T01",
    "T02",
    "T03",
    "T04",
    "T09",
    "T10",
    "T11",
    "T12",
    "G14_E23",
    "E09",
    "E10",
    "A04",
    "A05",
    "A06",
    "A07",
    "A08"
]
HH = [
    "G01",
    "G02",
    "G03",
    "G04",
    "G05",
    "G06",
    "G07",
    "G08",
    "LGSH01",
    "LGSH02",
    "LGSH03",
    "LGSH04",
    "G13",
    "G14",
    "G15",
    "G16",
    "G17",
    "G18"
]
TEMR_HH = [
    "H01",
    "H02",
    "H03",
    "H04",
    "H05",
    "H06",
    "H07",
    "H08",
    "G16_E24",
    "E02",
    "E06",
    "E11",
    "A09",
    "A10",
    "A11",
    "A12"
]
HS = [
    "S01",
    "S03",
    "S04",
    "S05",
    "S06",
    "S07",
    "S08",
    "S13",
    "S09",
    "S10",
    "S11",
    "S12",
    "S14",
    "S15",
    "S16",
    "S17"
]
TEMR_KG = [
    "K01",
    "K02",
    "K03",
    "K04",
    "K05",
    "K06",
    "K07",
    "K08",
    "G13_E22",
    "E01",
    "E03",
    "E04",
    "E05",
    "A01",
    "A02",
    "A03"
]

groups = {
    "NR": NR,
    "TEMR": TEMR,
    "HH": HH,
    "TEMR_HH": TEMR_HH,
    "HS": HS,
    "TEMR_KG": TEMR_KG
}
session = ['24wk', 'Week24']
valid_trials = {}
for group, subjects in groups.items():
    file_pattern = f"{root_dir}/{classification}/({'|'.join(subjects)})/({'|'.join(session)})/*.c3d"
    group_files = scandir_glob(root_dir, file_pattern)
    for file in group_files:
        trial = RatTrial.from_c3d_file(file)
        if not trial.validate_trial():
            continue
        subject = trial.subject_names or ["Unknown"]
        # Add the trial to the valid_trials dictionary in the format {classification}/{subject}/{session}/[{trial_name}]
        # Initialize nested dictionaries if they don't exist
        if trial.classification not in valid_trials:
            valid_trials[trial.classification] = {}
        if subject[0] not in valid_trials[trial.classification]:
            valid_trials[trial.classification][subject[0]] = {}
        if trial.session_name not in valid_trials[trial.classification][subject[0]]:
            valid_trials[trial.classification][subject[0]][trial.session_name] = []
        # Append the trial name to the list
        valid_trials[trial.classification][subject[0]][trial.session_name].append(trial.name)
        # Process the valid trial as needed
        logger.info(f"Processing trial: {file}")
        

